In [1]:
import numpy as np
import pandas as pd
from utilities import get_area_ratio, get_area_ratio_paper, get_year_dist

### Example 1: Testing implementation of ratio estimator using example data from Stehman (2014)

In [2]:
# Data from Stehman 2014

stratum = [1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4]
refClass = [1,1,1,1,1,3,2,1,2,3,1,2,2,2,2,2,1,1,2,2,3,3,3,3,3,4,4,2,2,1,4,4,4,4,4,4,4,3,3,2]
mapClass = [1,1,1,1,1,1,1,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,2,2,4,4,4,4,4,4,4,4,4,4]
areas = np.array([40000, 30000, 20000, 10000])

sample_info = np.vstack([stratum, mapClass, refClass])
counts = np.unique(stratum, return_counts=True)[1]
classes = np.unique(stratum, return_counts=True)[0]

In [3]:
# Class 1
for i in range(4):
    _class = i+1
    class_df = get_area_ratio_paper(areas, sample_info, counts, _class)
    print('Area of class {}: {}'.format(_class, class_df['area_prop']))
    print('Standard error of class {}: {}'.format(_class, class_df['se_prop']))


Area of class 1: 0    0.35
Name: area_prop, dtype: float64
Standard error of class 1: 0    0.082248
Name: se_prop, dtype: float64
Area of class 2: 0    0.34
Name: area_prop, dtype: float64
Standard error of class 2: 0    0.075853
Name: se_prop, dtype: float64
Area of class 3: 0    0.2
Name: area_prop, dtype: float64
Standard error of class 3: 0    0.06428
Name: se_prop, dtype: float64
Area of class 4: 0    0.11
Name: area_prop, dtype: float64
Standard error of class 4: 0    0.030722
Name: se_prop, dtype: float64


### Example 2: Estimating area of deforestation for one biennial period


In [10]:
# Inputs

# Map area and weights
total_area = 7545502177.858816

# Area weights
forest = 0.7561
non_forest = 0.0729
deg_only = 0.0285
def_only = 0.0554
disturbed = 0.0168
regrowth = 0.0051
buffer = 0.0651
area_weights = [forest, non_forest, deg_only, def_only, disturbed, regrowth, buffer]
area_list = total_area * np.array(area_weights)

# Sample counts
sample_counts = [1999, 200, 298, 298, 100, 99, 199]

# Reference data
ref_df = pd.read_csv('reference_samples.csv').fillna(0)

In [11]:
# Get reference and strata codes for the time period of interest
# Note this function is designed to return reference inputs for D/ND and deforestation only.
_strata, _ref, _map = get_year_dist(1996, 1997, ref_df)

sample_info = np.vstack([_strata, _map, _ref])
counts = np.unique(_strata, return_counts=True)[1]
classes = np.unique(_strata, return_counts=True)[0]

# Estimate area and uncertainty for deforestation and D/ND
deforestation = get_area_ratio(np.array(area_list), sample_info, np.array(sample_counts), 4)
dnd = get_area_ratio(np.array(area_list), sample_info, np.array(sample_counts), 3)

print('Deforestation in 1996-1997: {} km^2 (± {})'.format(deforestation.area_km2.round(-1).values[0], 
                                                          deforestation.se_km2.round(-1).values[0]))
print('D/ND in 1996-1997: {} km^2 (± {})'.format(dnd.area_km2.round(-1).values[0], 
                                                          dnd.se_km2.round(-1).values[0]))

Deforestation in 1996-1997: 35220.0 km^2 (± 6660.0)
D/ND in 1996-1997: 36140.0 km^2 (± 7790.0)
